In [5]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Get feature and label matrices

In [6]:
X_vector_len = 16000
train = SpeechList.get_train('../data/train/audio')
XY_train_valid = train.get_group_unknown_spectrogram_X_and_Y(X_vector_len=X_vector_len, spec_v='3', take_log=False, 
                                                             split_noise=True, n_fabricate_noise=1600, 
                                                             split=0.85)

# If using group_unknown, the output is: X_train_known, X_train_unknown, Y_train_known, Y_train_unknown, X_valid, Y_valid
# Otherwise: X_train, Y_train, X_valid, Y_valid

h:\python35\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


SKIPPED ../data/train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


h:\python35\lib\site-packages\librosa\filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


In [7]:

X_train, X_train_unknown, Y_train, Y_train_unknown, X_valid, Y_valid=XY_train_valid
print(X_train.shape)


(1701, 128, 71)


# Train

In [12]:
# Helper function
# def get_shape_after_skipping(initial_shape, strides_list):
#     h, w = initial_shape
#     for strides in strides_list:
#         h = math.ceil(h / strides[0])
#         w = math.ceil(w / strides[1])
#     return h, w

In [9]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.spectrogram_multi_lstm_random_input_modify_v2 import *

In [10]:
graph_name = 'SpectrogramMultiLSTMRandomInputModify2'  # Specify graph class name

g_cnfg = Config('graph_02_lower_lr_lr_decay')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]
g_cnfg.Y_vector_len = XY_train_valid[-1].shape[1]

g_cnfg.random_modify_brightness_max_delta = 0.25
g_cnfg.random_modify_contrast_lower = 0.2
g_cnfg.random_modify_contrast_upper = 1.8

g_cnfg.lstm_state_sizes = [1024, 1024]
g_cnfg.lstm_keep_prob = 0.75

g_cnfg.flat_hiddens = [512, 512, 50, 50]
g_cnfg.n_hidden1=512
g_cnfg.n_hidden2=512
g_cnfg.sigmoid_accuracy_threshold=0.1
#g_cnfg.conv1_keep_prob = 0.9
#g_cnfg.conv1_kernel_size = (10, 10)  # height, width
#g_cnfg.conv1_n_filters = 128
#g_cnfg.conv1_mp_size = (2, 1)
#g_cnfg.conv1_mp_strides = (2, 1)

#g_cnfg.conv2ds_keep_prob = 0.9
#g_cnfg.conv2ds_kernel_size = (3, 3)
#g_cnfg.conv2ds_strides = (1, 1)
#g_cnfg.conv2ds_n_filters = 128

#strides_list = [g_cnfg.conv1_mp_strides, g_cnfg.conv2ds_strides]
#g_cnfg.n_per_time_after_conv = get_shape_after_skipping((g_cnfg.X_img_h, g_cnfg.X_img_w), strides_list)[0] * g_cnfg.conv2ds_n_filters
#g_cnfg.n_per_time_after_reducing_conv = 128

g_cnfg.lr_initial = 0.01
g_cnfg.lr_decay_steps = 50000
g_cnfg.lr_decay_rate = 0.1  # 1.0 means no decay

t_cnfg = Config('run_01_group_unknown')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.random_pick_unknown = True
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.add_noise = (0.5, [s.spec_data for s in train.get_list_of_label(['_background_noise_'])])
t_cnfg.dropout_keep_prob = 0.75
t_cnfg.log_every = int(int(XY_train_valid[0].shape[0] / 11 * 12) / t_cnfg.batch_size / 2)
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [11]:
print(g_cnfg)
print(t_cnfg)
print('graph_name: ',graph_name)
GraphClass = globals()[graph_name]
print('graph_name : ',graph_name)
graph = GraphClass(g_cnfg)

graph.train_model(t_cnfg, XY_train_valid)

name: graph_02_lower_lr_lr_decay
X_img_h: 128
X_img_w: 71
Y_vector_len: 2
flat_hiddens: [512, 512, 50, 50]
lr_decay_rate: 0.1
lr_decay_steps: 50,000
lr_initial: 0.01
lstm_keep_prob: 0.75
lstm_state_sizes: [1024, 1024]
n_hidden1: 512
n_hidden2: 512
random_modify_brightness_max_delta: 0.25
random_modify_contrast_lower: 0.2
random_modify_contrast_upper: 1.8
sigmoid_accuracy_threshold: 0.1

name: run_01_group_unknown
add_noise: (0.5, 'list of length 2,002')
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.75
early_stopping_patience: 0.1
le: LabelEncoder()
log_dir: logs
log_every: 6
max_step: 100,000,000
n_ave_ll_valid: 8
random_pick_unknown: 1
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

graph_name:  SpectrogramMultiLSTMRandomInputModify2
graph_name :  SpectrogramMultiLSTMRandomInputModify2
self.X:  (?, 128, 71)
self.Y:  (?, 2)
pickle url  :  logs/SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_group_unknown.log
SpectrogramMultiLSTMRandomInp

X_batch:  (150, 128, 71)
X_batch:  [[-80.         -80.         -80.         ... -80.         -80.
  -80.        ]
 [-80.         -80.         -80.         ... -80.         -80.
  -80.        ]
 [-14.24895755  -6.40513465 -17.06170697 ... -12.0744168  -13.47523494
  -29.24423778]
 ...
 [-18.17848917 -19.61006108 -22.02183623 ... -22.57440831 -23.03530265
  -20.61820064]
 [-21.96448534 -20.50291564 -24.96051694 ... -21.91709458 -23.78297209
  -24.34123937]
 [-20.66086404 -21.95023046 -23.55572991 ... -23.48586079 -18.04966402
  -22.2206794 ]]
Y_batch:  (150, 2)
Y_batch:  [1. 0.]
pickle url  :  logs/SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_group_unknown.log
Epoch 1 Step 12 Best model saved @ 04/08/2018 12:59:25 [Ave valid] 0.679
Epoch 1 Step 12 ends @ 04/08/2018 12:59:25 [Train] 0.658, 97.3% [Valid] 91.8% [Ave valid] 0.679
X_batch:  (150, 128, 71)
X_batch:  [[-160.         -160.         -160.         ... -160.
  -160.         -160.        ]
 [-160.         

pickle url  :  logs/SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_group_unknown.log
Epoch 2 Step 24 Best model saved @ 04/08/2018 13:12:01 [Ave valid] 0.663
Epoch 2 Step 24 ends @ 04/08/2018 13:12:01 [Train] 0.631, 93.3% [Valid] 91.8% [Ave valid] 0.663
X_batch:  (150, 128, 71)
X_batch:  [[-160.         -160.         -160.         ... -160.
  -160.         -160.        ]
 [-160.         -160.         -160.         ... -160.
  -160.         -160.        ]
 [ -33.97076819  -42.29044679  -39.38466923 ...  -48.63222005
   -41.21467985  -38.41412872]
 ...
 [ -42.31733669  -53.32206964  -47.64145048 ...  -48.41659799
   -46.69459285  -50.16444542]
 [ -48.55732226  -50.33942784  -43.56796743 ...  -45.02822709
   -43.7114746   -46.61289167]
 [ -47.03897102  -49.35890898  -45.33406305 ...  -43.75410491
   -49.86018068  -43.22223904]]
Y_batch:  (150, 2)
Y_batch:  [1. 0.]


KeyboardInterrupt: 